In [ ]:
import os
import sys
import cv2
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import skimage
from PIL import Image
import imutils
import imgaug.augmenters as iaa

%matplotlib inline

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

import PIL.Image
import io
import os.path as osp

def apply_exif_orientation(image):
    try:
        exif = image._getexif()
    except AttributeError:
        exif = None

    if exif is None:
        return image

    exif = {
        PIL.e.TAGS[k]: v
        for k, v in exif.items()
        if k in PIL.ExifTags.TAGS
    }

    orientation = exif.get("Orientation", None)

    if orientation == 1:
        # do nothing
        return image
    elif orientation == 2:
        # left-to-right mirror
        return PIL.ImageOps.mirror(image)
    elif orientation == 3:
        # rotate 180
        return image.transpose(PIL.Image.ROTATE_180)
    elif orientation == 4:
        # top-to-bottom mirror
        return PIL.ImageOps.flip(image)
    elif orientation == 5:
        # top-to-left mirror
        return PIL.ImageOps.mirror(image.transpose(PIL.Image.ROTATE_270))
    elif orientation == 6:
        # rotate 270
        return image.transpose(PIL.Image.ROTATE_270)
    elif orientation == 7:
        # top-to-right mirror
        return PIL.ImageOps.mirror(image.transpose(PIL.Image.ROTATE_90))
    elif orientation == 8:
        # rotate 90
        return image.transpose(PIL.Image.ROTATE_90)
    else:
        return image


def load_image_file(filename):
    try:
        image_pil = PIL.Image.open(filename)
    except IOError:
        logger.error("Failed opening image file: {}".format(filename))
        return

    # apply orientation to image according to exif
    image_pil = apply_exif_orientation(image_pil)

    with io.BytesIO() as f:
        ext = osp.splitext(filename)[1].lower()
        if ext in [".jpg", ".jpeg"]:
            format = "JPEG"
        else:
            format = "PNG"
        image_pil.save(f, format=format)
        f.seek(0)
        return f.read()    

In [ ]:
class_names = ['stone']
numbers = 5000
height = 1275
width = 850

grid_size = 270
grid_w = int(width / grid_size)
grid_h = int(height / grid_size)

grid_num = [i for i in range(grid_w * grid_h)]
            
CROP_PATH = "./stone_sample"
crop_folder_list = os.listdir(CROP_PATH)

img_num = 0

for number in range(0, numbers):

    tray = np.zeros((height, width, 4))
    CROP_FOLDER_PATH = CROP_PATH
    crop_img_list = crop_folder_list
    
    stone_num = grid_w * grid_h
    
    if len(crop_img_list) < stone_num:
        stone_num = len(crop_img_list)
    
    stone_list = random.sample(crop_img_list, stone_num)
    random.shuffle(stone_list)
    
    stone_grid_num_list = random.sample(grid_num, stone_num)
    shapes = []
    history_images = []
    
    for idx,one_stone in enumerate(stone_list):

        # stone_img load
        one_stone_path = os.path.join(CROP_FOLDER_PATH, one_stone)

        stone_img = cv2.imread(one_stone_path, cv2.IMREAD_UNCHANGED)
        
        
        stone_img = Image.fromarray(stone_img)
#         stone_img_con = cv2.cvtColor(stone_img, cv2.COLOR_BGRA2BGR)
        stone_img = stone_img.convert("RGBA")
        datas = stone_img.getdata()

        newData = []
        for item in datas:
            if item[0] == 255 and item[1] == 255 and item[2] == 255:
                newData.append((item[0], item[1], item[2], 0))
            else:
                newData.append(item)

        stone_img.putdata(newData)
        stone_img = np.asarray(stone_img)
        
        
        rand_ratio = random.uniform(0.9,1.2)
        rand_rotate = random.choice([0, 90, 180, 270])
        
        # Augmentation
        stone_img = cv2.resize(stone_img, dsize=(0, 0), fx=rand_ratio, fy=rand_ratio, interpolation=cv2.INTER_AREA)

#         # Show the output image
#         cv2.imshow('Output', stone_img)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        
        if number % 10 == 0:
            stone_img = cv2.blur(stone_img,(5,5))
            
        
        
        if number % 11 == 0:
            b, g, r, a = cv2.split(stone_img)
            stone_img = cv2.merge((b, g, r))
            aug_tem = iaa.ChangeColorTemperature((4000, 4010))
            stone_img = aug_tem(image=stone_img)
            stone_img = cv2.merge((stone_img, a))
        elif number % 12 == 0:
            b, g, r, a = cv2.split(stone_img)
            stone_img = cv2.merge((b, g, r))
            aug_tem = iaa.ChangeColorTemperature((10000, 10010))
            stone_img = aug_tem(image=stone_img)
            stone_img = cv2.merge((stone_img, a))

        elif number % 5 == 0:
            b, g, r, a = cv2.split(stone_img)
            stone_img = cv2.merge((b, g, r))
            aug_bright = iaa.WithBrightnessChannels(iaa.Add((-50, 50)))
            stone_img = aug_bright(image=stone_img)
            stone_img = cv2.merge((stone_img, a))
        
            
        
        
#         stone_img = convert_temp(stone_img, rand_temp)
        stone_img = np.array(stone_img)
        
        stone_w = stone_img.shape[1]
        stone_h = stone_img.shape[0]
        # print(stone_w, stone_h)

        # cal x, y with grid number
        stone_grid_num = stone_grid_num_list[idx]
        start_x = stone_grid_num % grid_w * grid_size
        start_y = stone_grid_num // grid_w * grid_size

        # outline 
        if start_x + stone_w > 850:
            stone_w = 850 - start_x
        if start_y + stone_h > 1275:
            stone_h = 1275 - start_y

        # put stone to tray / tmp_tray
        stone_area = np.where(stone_img[:stone_h, :stone_w, 3] == 255)
        

#         tmp_tray = np.zeros((2550, 1700, 4))
        tmp_tray = np.zeros((stone_h,stone_w,4))
        tmp_tray[stone_area[0],stone_area[1],:] = stone_img[stone_area]
        
        
        # tmp_tray -> contour -> annot_x,y
        tmp_tray = (tmp_tray).astype(np.uint8)
        tmp_tray = cv2.cvtColor(tmp_tray, cv2.COLOR_BGRA2BGR)
        tmp_tray = cv2.cvtColor(tmp_tray, cv2.COLOR_BGR2GRAY)
#         ret, tmp_tray = cv2.threshold(tmp_tray,0,255, cv2.THRESH_BINARY)
        contours, hierachy = cv2.findContours(tmp_tray, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_KCOS)
        c = max(contours, key = cv2.contourArea)
        
#         print(np.array(contours).shape[0])
#         print(np.array(c).shape)
        
#         out_line = np.squeeze(contours[0])
        out_line = np.squeeze(c)

        
        annot_x = []
        annot_y = []
#         if np.array(contours).shape[0] != 1:          
            
#             if idx == 0:
#                 flag_for_idx = 1
#             continue
        challenge_image = np.zeros((height, width, 4))
        challenge_image[start_y + stone_area[0],start_x + stone_area[1],:] = 100
#         print("chall")
#         plt.imshow(challenge_image.astype(np.uint8))
#         plt.show()
        history_image = np.zeros((height, width, 4))
        history_image[start_y + stone_area[0],start_x + stone_area[1],:] = 100
#         print("hist")
#         plt.imshow(history_image.astype(np.uint8))
#         plt.show()
        
        history_images.append(history_image)
        
        for x in range(len(history_images)):
            overlap_ratio = 0
            if idx == 0:
                break
            one_history = history_images[x]
            challenge_image = one_history + challenge_image
            

            overlap_ratio = (len(np.where(challenge_image > 150)[1]) / len(np.where(one_history < 150)[1])) * 1000
            

            if overlap_ratio >= 100:
                idx += 1
                print("opverlayed")
                break
                
        if overlap_ratio >= 100:
            continue
        tray[start_y + stone_area[0],start_x + stone_area[1],:] = stone_img[stone_area]
        
        
#         try:
        for x,y in out_line:
            annot_x.append(x)
            annot_y.append(y)
#         except:
#             print(img_num)
#             print(np.array(contours).shape)
#             break
        
        annot_min_x = np.min(annot_x)
        annot_min_y = np.min(annot_y)
        
        points = []
        for x,y in list(zip(annot_x, annot_y)):
            point = list(zip([start_x+x-annot_min_x], [start_y+y-annot_min_y]))

            points.append(point[0])

        shapes_info = [dict(label = "stone",
               points = points,
               group_id = None,
               shape_type = "polygon",
               flags = {}
                )
            ]
        shape = dict(shapes = shapes_info)
        otherData = {}
#         shapes.append(shapes_info)

        
        synthetic_imagename = "./segment_dataset_aug_2/synthetic_image_%d.jpg" %img_num
        synthetic_jsonname = "./segment_dataset_aug_2/synthetic_image_%d.json" %img_num


        

        import base64

        start = dict(
                version="4.4.0",
                flags={},
                shapes=shapes_info
            )

        if idx == 0:
                with open(synthetic_jsonname, "w") as json_file:
                    json.dump(start, json_file, ensure_ascii=False, indent=2, cls=NumpyEncoder)
                idx += 1
#                 flag_for_idx = 0
                

        else:
                try:
                    with open(synthetic_jsonname) as f:
                        read = json.load(f) 
                        temp = read['shapes']

                        temp.append(shapes_info[0])


                    with open(synthetic_jsonname, "w") as f:
                        json.dump(read, f, ensure_ascii=False, indent=2, cls=NumpyEncoder)
                    idx += 1
                except:
                    
                    print("can't create json")
                    
    plt.imshow(tray.astype(np.uint8))
    plt.show()
    img_num += 1
    tray = (tray).astype(np.uint8)
    tray = cv2.cvtColor(tray, cv2.COLOR_BGRA2RGB)
    cv2.imwrite(synthetic_imagename, tray)
    imageData = load_image_file(synthetic_imagename)
    imageData = base64.b64encode(imageData).decode("utf-8")    

#     json_write = dict(
#             version="4.4.0",
#             flags={},
#             shapes=shapes,
#             imagePath = synthetic_imagename,
#             imageData = imageData,
#             imageHeight=height,
#             imageWidth=width
#         )
#     with open(synthetic_jsonname, "w") as json_file: 
#         json.dump(json_write, json_file, ensure_ascii=False, indent=2, cls=NumpyEncoder)

# plt.imshow(tray)
# plt.show()

    with open(synthetic_jsonname) as f:
        read = json.load(f)
    read["imagePath"] = synthetic_imagename
    read["imageData"] = str(imageData)
    read["imageHeight"] = 1275
    read["imageWidth"] = 850

    with open(synthetic_jsonname, "w") as f:
        json.dump(read, f, ensure_ascii=False, indent=2, cls=NumpyEncoder)
